In [11]:
import pandas as pd
import numpy as np

import torch
import torch.optim as optim
import torch.nn.functional as F

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit= pd.read_csv('submit.csv')

In [3]:
train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Diabetes
0,1,95,60,18,58,23.9,0.260,22,0
1,5,105,72,29,325,36.9,0.159,28,0
2,0,135,68,42,250,42.3,0.365,24,1
3,4,131,68,21,166,33.1,0.160,28,0
4,1,103,30,38,83,43.3,0.183,33,0
...,...,...,...,...,...,...,...,...,...
532,5,139,64,35,140,28.6,0.411,26,0
533,1,96,122,0,0,22.4,0.207,27,0
534,10,101,86,37,0,45.6,1.136,38,1
535,0,141,0,0,0,42.4,0.205,29,1


In [4]:
train_x = train.drop(['Diabetes'], axis =1)
train_y = pd.DataFrame(train.Diabetes)

test_x = test

In [5]:
train_x_tensor = torch.FloatTensor(train_x.values) #537,8
train_y_tensor = torch.FloatTensor(train_y.values) #537,1

test_x_tensor = torch.FloatTensor(test_x.values)

In [6]:
W = torch.zeros((8,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [7]:
optimizer = optim.SGD([W,b], lr=1e-4)
total_epoch= 1000000
for epoch in range(1,total_epoch+1):
    hypothesis = F.sigmoid(train_x_tensor.matmul(W) + b)
    
    loss = torch.nn.BCELoss()
    cost = loss(hypothesis, train_y_tensor)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, total_epoch, cost.item()
        ))

C:\Users\yoons\anaconda3\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 100000/1000000 Cost: 0.603285
Epoch 200000/1000000 Cost: 0.589933
Epoch 300000/1000000 Cost: 0.578006
Epoch 400000/1000000 Cost: 0.567298
Epoch 500000/1000000 Cost: 0.557735
Epoch 600000/1000000 Cost: 0.549164
Epoch 700000/1000000 Cost: 0.541442
Epoch 800000/1000000 Cost: 0.534546
Epoch 900000/1000000 Cost: 0.528381
Epoch 1000000/1000000 Cost: 0.522740


In [8]:
predict_hypothesis = F.sigmoid(test_x_tensor.matmul(W) + b)
predict = predict_hypothesis>= torch.FloatTensor([0.5])

In [9]:
submit.Diabetes = predict.detach().numpy().astype('int')

In [10]:
submit.to_csv('2021_09_29_03.csv', index=False) #0.601884 #0.522740